In [1]:
import requests
import json
import datetime
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
#ローカルにおいてあるconfigファイルを読み出して、認証情報を取得しておく
import sys
sys.path.append("../")
from instaapi.local_config import * 

In [3]:
version = 'v16.0'

In [4]:
def basic_info():
    # 初期
    config = dict()
    #アクセストークン※facebook開発アプリから取得
    config["access_token"]         = ACCESS_TOKEN
    #アプリID※facebook開発アプリから取得
    config["app_id"]               = APP_ID
    #アプリシークレット※facebook開発アプリから取得
    config["app_secret"]           = APP_SECRET
    #インスタグラムビジネスアカウントID※facebook開発アプリから取得
    config['instagram_account_id'] = INSTAGRAM_ACCOUNT_ID
    #APIバージョン
    config["version"]              = version
    # 【修正不要】graphドメイン
    config["graph_domain"]         = 'https://graph.facebook.com/'
    # 【修正不要】エンドポイント
    config["endpoint_base"]        = config["graph_domain"]+config["version"] + '/'
    # 出力
    return config

In [5]:
# リクエスト
params   = basic_info()       # リクエストパラメータ
#response = debugAT(params)    # レスポンス


In [7]:
'''各投稿のinformation(基本的な情報)を取得'''


import requests
#import pandas as pd
#pd.set_option('display.max_rows', None)

# アクセス情報
business_account_id = INSTAGRAM_ACCOUNT_ID
token = ACCESS_TOKEN
username = USER_NAME
fields = 'name,username,biography,follows_count,followers_count,media_count'
media_fields = 'timestamp,permalink,media_url,like_count,comments_count,caption'
period = 'day'


# ユーザー情報を取得する
def user_info(business_account_id=INSTAGRAM_ACCOUNT_ID,token=ACCESS_TOKEN,username=username,fields=fields):
    request_url = "https://graph.facebook.com/"+version+"/{business_account_id}?fields=business_discovery.username({username}){{{fields}}}&access_token={token}".format(business_account_id=business_account_id,username=username,fields=fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    return response.json()['business_discovery']

#print(user_info(business_account_id,token,username,fields))

# メディア情報を取得する
def user_media_info(business_account_id,token,username,media_fields):
    all_response = []

    request_url = "https://graph.facebook.com/"+version+"/{business_account_id}?fields=business_discovery.username({username}){{media{{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    result = response.json()['business_discovery']
    
    all_response.append(result['media']['data'])
    
    # 過去分がある場合は過去分全て取得する(1度に取得できる件数は25件)
    if 'after' in result['media']['paging']['cursors'].keys():
        next_token = result['media']['paging']['cursors']['after']
        while next_token is not None:
            request_url = "https://graph.facebook.com/"+version+"/{business_account_id}?fields=business_discovery.username({username}){{media.after({next_token}){{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token,next_token=next_token)
#             print(request_url)
            response = requests.get(request_url)
            result = response.json()['business_discovery']
            all_response.append(result['media']['data'])
            if 'after' in result['media']['paging']['cursors'].keys():
                next_token = result['media']['paging']['cursors']['after']
            else:
                next_token = None
    
    return all_response

result = user_media_info(business_account_id,token,username,media_fields)

'''結果をデータフレームに格納'''
df_media_info = pd.DataFrame(result[0])
for noc in np.arange(1, len(result)):
    output_per_call = pd.DataFrame(result[noc])
    df_media_info = pd.concat([df_media_info,output_per_call], ignore_index=True)
    
df_media_info


,timestamp,permalink,media_url,like_count,comments_count,caption,id
0,2023-06-09T09:30:00+0000,https://www.instagram.com/p/CtTg1i4ppwo/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,26,0,---\n------\n◽︎Jacket : #lardini \n◽︎Shirts : ...,17996687311808013
1,2023-06-07T09:30:00+0000,https://www.instagram.com/p/CtLyb6drMPn/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,73,3,---\n------\n◽︎Jacket : #beamsf \n◽︎Shirts : #...,18009590089669016
2,2023-06-05T09:30:00+0000,https://www.instagram.com/p/CtGoyQHLLtR/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,63,3,---\n------\n◽︎Jacket : #lardini \n◽︎Shirts : ...,18031813213503721
3,2023-05-31T09:30:00+0000,https://www.instagram.com/p/Cs0nUVaryCi/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,72,3,---\n------\n◽︎Jacket : #beamsf \n◽︎Shirts : #...,18005121412787276
4,2023-05-29T09:30:00+0000,https://www.instagram.com/p/Cs0nUk8L20t/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,65,4,---\n------\n◽︎Jacket : #unitedarrows \n◽︎Shir...,17880368870857555
...,...,...,...,...,...,...,...
124,2022-01-13T13:16:58+0000,https://www.instagram.com/p/CYq-71wpmv4/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,128,0,リモートワークが続きます。部屋は仕事環境としても居住環境としても良いものに整えたいですね。\...,17906032391509098
125,2021-12-07T12:01:02+0000,https://www.instagram.com/p/CXLk1XmpNUx/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,63,1,coat : #barbour #barbourbedale \njacket : #edi...,18014576203356847
126,2021-12-05T13:17:15+0000,https://www.instagram.com/p/CXGj9v4pFjs/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,44,0,#poloralphlauren \n#damico #andreadamico \n\n...,18206637583136542
127,2021-12-01T11:55:03+0000,https://www.instagram.com/p/CW8HYTwpjgV/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,72,3,jacket : #lardini \nshirt : #kamakurashirts \n...,18272253838038998


In [8]:

'''ここで全投稿のmedia idのリストも作っておく'''

list_media_id = []
for noc in np.arange(len(result)):
    for nop in np.arange(len(result[noc])):
        media_id = result[noc][nop]['id']
        list_media_id.append(media_id)
print('最初の10個だけ表示：',list_media_id[:10])
print('全投稿数：',len(list_media_id))

最初の10個だけ表示： ['17996687311808013', '18009590089669016', '18031813213503721', '18005121412787276', '17880368870857555', '18010434379633055', '17962398962524619', '18265673275182451', '18065947474383078', '17874336896877998']
全投稿数： 129


In [9]:
'''投稿ごとのさらに詳細なinsight(reach, saved, impressions, enagegement)を取得する
    ※これは各投稿IDごとにAPIコールが必要なため、処理に少し時間がかかる。改善方法を検討中'''

"""
    ***********************************************************************************
    【APIエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-media-id}/insights?metric={metric}
    ***********************************************************************************
    """
#media IDからインサイトを取得する関数
def media_insight(media_id):
    
    '''リクエスト先のurl作成'''
    metric_for_url = ''
    for mt in metric:
        metric_for_url += mt + '%2C'
    metric_for_url = metric_for_url.rstrip('%2C')
    request_url = params['endpoint_base'] + media_id + '/insights?access_token=' + params[
        'access_token'] + '&metric=' + metric_for_url
    
    response = requests.get(request_url).json()['data']
    response_reshape = dict()
    response_reshape['id'] = media_id
    response_reshape['reach'] = response[0]['values'][0]['value']
    response_reshape['impressions'] = response[1]['values'][0]['value']
    response_reshape['saved'] = response[2]['values'][0]['value']
    response_reshape['engagement'] = response[3]['values'][0]['value']
    
    return response_reshape


metric = ['reach','impressions','saved','engagement']

#全てのmediaIDでインサイトを取得する
results = []
for nom in np.arange(len(list_media_id)):
    media_id = list_media_id[nom]
    out = media_insight(media_id)
    results.append(out)

In [10]:
df_media_insight = pd.DataFrame(results)
df_media_insight

,id,reach,impressions,saved,engagement
0,17996687311808013,154,172,0,26
1,18009590089669016,271,277,3,79
2,18031813213503721,238,250,1,67
3,18005121412787276,203,219,1,76
4,17880368870857555,214,232,0,69
...,...,...,...,...,...
124,17906032391509098,479,524,0,128
125,18014576203356847,469,503,0,64
126,18206637583136542,294,316,0,44
127,18272253838038998,337,370,0,75


In [ ]:
'''結合してカラムの順番整理'''

df_media = pd.merge(df_media_info,df_media_insight, how='inner', on='id')
df_media = df_media.reindex(columns=[
    'id', 'timestamp','permalink','media_url','like_count','saved','reach','impressions','engagement','caption'])
df_media

In [ ]:
#csvに書き出して保存
today = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')[:8]
df_media.to_csv('../data/output/media_insight_'+today+'.csv')

In [ ]:
'''直近30日間のアカウントステータスを取得する'''
# ※30日は遡れる日数の上限

import datetime
#今日と28日さかのぼった日付をyyyy/mm/ddの形式で取得する 標準時に設定
delta_days = 30
today = datetime.datetime.utcnow()
date_delta = datetime.datetime.utcnow() - datetime.timedelta(days=delta_days)

yyyymmdd_td = '{yyyy}/{mm}/{dd}'.format(yyyy=today.year,mm=today.month,dd=today.day)
yyyymmdd_delta = '{yyyy}/{mm}/{dd}'.format(yyyy=date_delta.year,mm=date_delta.month,dd=date_delta.day)

print('今日の日付：',yyyymmdd_td)
print('遡った日付：',yyyymmdd_delta)

In [18]:
# エンドポイントURLの作成
metric = ['follower_count','impressions','profile_views','reach']
metric_for_url = ''
for mt in metric:
    metric_for_url += mt + '%2C'
metric_for_url = metric_for_url.rstrip('%2C')

request_url = params['endpoint_base'] + params['instagram_account_id'] + '/insights?access_token=' + params[
    'access_token'] + '&metric=' + metric_for_url + '&period=day'+ '&since=' + yyyymmdd_delta + '&until=' + yyyymmdd_td
response = requests.get(request_url).json()['data']

In [19]:
list_endtimes = []
list_impressions = []
list_follower_count = []
list_reach = []
list_profile_views = []

for day_n in range(delta_days):
    list_endtimes.append(response[0]['values'][day_n]['end_time'])
    list_follower_count.append(response[0]['values'][day_n]['value'])
    list_impressions.append(response[1]['values'][day_n]['value'])
    list_profile_views.append(response[2]['values'][day_n]['value'])
    list_reach.append(response[3]['values'][day_n]['value'])
    
result = dict()
result['endtime'] = list_endtimes
result['follower_count'] = list_follower_count
result['profile_views'] = list_profile_views
result['impressions'] = list_impressions
result['reach'] = list_reach


In [20]:
#データフレームとして格納
df_account_status_in30days = pd.DataFrame(result)
df_account_status_in30days

#時系列で並び替え
df_account_status_in30days = df_account_status_in30days.sort_values(by='endtime', ascending=False, ignore_index=True)

In [21]:
#csvに書き出して保存
today = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')[:8]
df_account_status_in30days.to_csv('../data/output/account_status_'+today+'.csv')

In [14]:
df_media_info

,timestamp,permalink,media_url,like_count,comments_count,caption,id
0,2023-06-09T09:30:00+0000,https://www.instagram.com/p/CtTg1i4ppwo/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,26,0,---\n------\n◽︎Jacket : #lardini \n◽︎Shirts : ...,17996687311808013
1,2023-06-07T09:30:00+0000,https://www.instagram.com/p/CtLyb6drMPn/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,73,3,---\n------\n◽︎Jacket : #beamsf \n◽︎Shirts : #...,18009590089669016
2,2023-06-05T09:30:00+0000,https://www.instagram.com/p/CtGoyQHLLtR/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,63,3,---\n------\n◽︎Jacket : #lardini \n◽︎Shirts : ...,18031813213503721
3,2023-05-31T09:30:00+0000,https://www.instagram.com/p/Cs0nUVaryCi/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,72,3,---\n------\n◽︎Jacket : #beamsf \n◽︎Shirts : #...,18005121412787276
4,2023-05-29T09:30:00+0000,https://www.instagram.com/p/Cs0nUk8L20t/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,65,4,---\n------\n◽︎Jacket : #unitedarrows \n◽︎Shir...,17880368870857555
...,...,...,...,...,...,...,...
124,2022-01-13T13:16:58+0000,https://www.instagram.com/p/CYq-71wpmv4/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,128,0,リモートワークが続きます。部屋は仕事環境としても居住環境としても良いものに整えたいですね。\...,17906032391509098
125,2021-12-07T12:01:02+0000,https://www.instagram.com/p/CXLk1XmpNUx/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,63,1,coat : #barbour #barbourbedale \njacket : #edi...,18014576203356847
126,2021-12-05T13:17:15+0000,https://www.instagram.com/p/CXGj9v4pFjs/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,44,0,#poloralphlauren \n#damico #andreadamico \n\n...,18206637583136542
127,2021-12-01T11:55:03+0000,https://www.instagram.com/p/CW8HYTwpjgV/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,72,3,jacket : #lardini \nshirt : #kamakurashirts \n...,18272253838038998
